In [29]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [17]:
anime_data = pd.read_csv('dataset/anime.csv')
anime_df = anime_data[['MAL_ID', 'Name','Genres','Completed']]
anime_df.head()

,MAL_ID,Name,Genres,Completed
0,1,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",718161
1,5,Cowboy Bebop: Tengoku no Tobira,"Action, Drama, Mystery, Sci-Fi, Space",208333
2,6,Trigun,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen",343492
3,7,Witch Hunter Robin,"Action, Mystery, Police, Supernatural, Drama, ...",46165
4,8,Bouken Ou Beet,"Adventure, Fantasy, Shounen, Supernatural",7314


In [5]:
rating_popular_anime = pd.read_csv('dataset/top_anime_unsupervised_use.csv')

In [12]:
anime_matrix_UII=rating_popular_anime.pivot_table(index='user_id',columns='MAL_ID',values='rating')
anime_matrix_UII.head()

MAL_ID,1,5,6,7,8,15,16,17,18,19,...,47307,47398,47402,47403,47410,47614,47616,47618,48177,48456
user_id,,,,,,,,,,,,,,,,,,,,,
478,NaN,NaN,NaN,5.0,NaN,8.0,7.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
781,7.0,NaN,10.0,10.0,NaN,NaN,7.0,NaN,8.0,10.0,...,6.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
853,9.0,10.0,8.0,7.0,NaN,9.0,NaN,8.0,9.0,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
890,9.0,9.0,9.0,NaN,NaN,NaN,NaN,NaN,8.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
912,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN


In [13]:
test_user_rating = anime_matrix_UII[1]
test_user_rating.head()

user_id
478    NaN
781    7.0
853    9.0
890    9.0
912    NaN
Name: 1, dtype: float64

In [14]:
similar_to_test_amime = anime_matrix_UII.corrwith(test_user_rating)
similar_to_test_amime.head()

/Users/siyuchen/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:2634: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/Users/siyuchen/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:2493: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


MAL_ID
1    1.000000
5    0.671262
6    0.441822
7    0.275592
8    0.248126
dtype: float64

In [34]:
corr_test = pd.DataFrame(similar_to_test_amime, columns=['correlation'])
corr_test.dropna(inplace=True)
corr_test.reset_index(inplace=True)
corr_test.head()

,MAL_ID,correlation
0,1,1.000000
1,5,0.671262
2,6,0.441822
3,7,0.275592
4,8,0.248126


In [35]:
corr_test = pd.merge(corr_test, anime_df, on="MAL_ID")
corr_test.head()

,MAL_ID,correlation,Name,Genres,Completed
0,1,1.000000,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",718161
1,5,0.671262,Cowboy Bebop: Tengoku no Tobira,"Action, Drama, Mystery, Sci-Fi, Space",208333
2,6,0.441822,Trigun,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen",343492
3,7,0.275592,Witch Hunter Robin,"Action, Mystery, Police, Supernatural, Drama, ...",46165
4,8,0.248126,Bouken Ou Beet,"Adventure, Fantasy, Shounen, Supernatural",7314


In [36]:
corr_test[corr_test['Completed'] > 10000].sort_values(by='correlation', ascending=False).head(10)

,MAL_ID,correlation,Name,Genres,Completed
0,1,1.000000,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",718161
1,5,0.671262,Cowboy Bebop: Tengoku no Tobira,"Action, Drama, Mystery, Sci-Fi, Space",208333
3448,4037,0.528822,Cowboy Bebop: Yose Atsume Blues,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",20913
23,43,0.446266,Koukaku Kidoutai,"Action, Mecha, Police, Psychological, Sci-Fi, ...",325682
181,205,0.445135,Samurai Champloo,"Action, Adventure, Comedy, Historical, Samurai...",551621
2,6,0.441822,Trigun,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen",343492
1329,1471,0.439428,City Hunter 2,"Action, Comedy, Mystery, Shounen",10909
8201,20057,0.432818,Space☆Dandy,"Sci-Fi, Space, Comedy",135822
8884,23327,0.432198,Space☆Dandy 2nd Season,"Space, Comedy, Sci-Fi",85493
426,457,0.428542,Mushishi,"Adventure, Slice of Life, Mystery, Historical,...",235371


In [39]:
def fit_knn_item_based(input_MAL_ID):
    user_rating_input = anime_matrix_UII[input_MAL_ID]
    print()
    similar_to_test_amime = anime_matrix_UII.corrwith(test_user_rating)
    corr_test = pd.DataFrame(similar_to_test_amime, columns=['correlation'])
    corr_test.dropna(inplace=True)
    corr_test.reset_index(inplace=True)
    corr_test = pd.merge(corr_test, anime_df, on="MAL_ID")
    recommendation = corr_test[corr_test['Completed'] > 10000].sort_values(by='correlation', ascending=False).head(11)
    recommendation.drop(recommendation.index[recommendation['MAL_ID'] == input_MAL_ID], inplace=True)
    return recommendation

recommendation_test = fit_knn_item_based(3786)
recommendation_test

,MAL_ID,correlation,Name,Genres,Completed
0,1,1.000000,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",718161
1,5,0.671262,Cowboy Bebop: Tengoku no Tobira,"Action, Drama, Mystery, Sci-Fi, Space",208333
3448,4037,0.528822,Cowboy Bebop: Yose Atsume Blues,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",20913
23,43,0.446266,Koukaku Kidoutai,"Action, Mecha, Police, Psychological, Sci-Fi, ...",325682
181,205,0.445135,Samurai Champloo,"Action, Adventure, Comedy, Historical, Samurai...",551621
2,6,0.441822,Trigun,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen",343492
1329,1471,0.439428,City Hunter 2,"Action, Comedy, Mystery, Shounen",10909
8201,20057,0.432818,Space☆Dandy,"Sci-Fi, Space, Comedy",135822
8884,23327,0.432198,Space☆Dandy 2nd Season,"Space, Comedy, Sci-Fi",85493
426,457,0.428542,Mushishi,"Adventure, Slice of Life, Mystery, Historical,...",235371
